In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out import create_dir
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.ae_templates import conv_architecture_ala_nips_17, default_train_params_ala_nips_17
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only, encoder_with_convs_and_symmetry_new
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder

from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.shape_net import pc_loader as snc_loader
from tf_lab.data_sets.shape_net import snc_category_to_synth_id
from tflearn.layers.conv import avg_pool_1d
from pc_completions.evaluation import basic_comletion_measures

from tflearn.layers.conv import conv_1d
from tflearn.layers.normalization import batch_normalization
from testing_encoder_decoder import decoder_with_convs_only

from tf_lab.fundamentals.layers import conv_1d_tranpose

In [14]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
n_pc_points = 2048
top_data_dir = '/orions4-zfs/projects/optas/DATA/'
top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))

In [5]:
do_training = True
load_pretrained_model = False
load_epoch = None
random_seed = 42
loss = 'chamfer'
training_epochs = 250
batch_size = 50
learning_rate = 0.0005

In [6]:
train_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/train.txt')
val_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/val.txt')
test_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/03001627/85_5_10/test.txt')

splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)

tr_files = splitter.load_splits(train_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(tr_files, n_threads=20, loader=snc_loader, verbose=True)
train_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

val_files = splitter.load_splits(val_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(val_files, n_threads=20, loader=snc_loader, verbose=True)
val_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

test_files = splitter.load_splits(test_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(test_files, n_threads=20, loader=snc_loader, verbose=True)
test_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.


In [26]:
from functools import partial
from tflearn.layers.conv import avg_pool_1d, max_pool_1d

b_norm = True

deconv_op = partial(conv_1d_tranpose, batch_size=batch_size)

encoder_args = {'n_filters': [64, 64, 128, 128, 256, 256, 32],
                'filter_sizes': [9, 8, 8, 5, 3, 3, 2],
                'strides': [1, 1, 1, 1, 1, 1, 1],
                'b_norm': b_norm,
                'pool': avg_pool_1d,
                'pool_sizes': [4, 4, 4, 4, 2, 2, 2],
                'symmetry': None,
                'padding': 'same'
                }

decoder_args = {'n_filters': [256, 128, 128, 64, 64, 3],
                'filter_sizes': [3, 3, 5, 5, 5, 6],
                'strides': [2, 2, 4, 4, 4, 8],
                'b_norm': b_norm,
                'non_linearity': tf.nn.relu,
                'b_norm_finish': False,
                'conv_op': deconv_op
                }

layer = tf.placeholder(tf.float32, [None, n_pc_points, 3])
layer = encoder_with_convs_and_symmetry_new(layer, **encoder_args)
decoder_with_convs_only(layer, **decoder_args)

conv params =  1792
bnorm params =  128
Tensor("AvgPool1D_6/Squeeze:0", shape=(?, 512, 64), dtype=float32) 32768
conv params =  32832
bnorm params =  128
Tensor("AvgPool1D_7/Squeeze:0", shape=(?, 128, 64), dtype=float32) 8192
conv params =  65664
bnorm params =  256
Tensor("AvgPool1D_8/Squeeze:0", shape=(?, 32, 128), dtype=float32) 4096
conv params =  82048
bnorm params =  256
Tensor("AvgPool1D_9/Squeeze:0", shape=(?, 8, 128), dtype=float32) 1024
conv params =  98560
bnorm params =  512
Tensor("AvgPool1D_10/Squeeze:0", shape=(?, 4, 256), dtype=float32) 1024
conv params =  196864
bnorm params =  512
Tensor("AvgPool1D_11/Squeeze:0", shape=(?, 2, 256), dtype=float32) 512
conv params =  16416
bnorm params =  64
Tensor("AvgPool1D_12/Squeeze:0", shape=(?, 1, 32), dtype=float32) 32
Building Decoder
25088
98560
82176
41088
20608
1158


<tf.Tensor 'decoder_conv_layer_5_1/BiasAdd:0' shape=(50, 2048, 3) dtype=float32>

In [27]:
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings/temp')

conf = Conf(
            n_input = [n_pc_points, 3],
            loss = loss,
            training_epochs = training_epochs,
            batch_size = batch_size,
            denoising = False,
            learning_rate = learning_rate,
            train_dir = train_dir,             
            loss_display_step = 1,
            saver_step = None,
            z_rotate = False,
            encoder = encoder_with_convs_and_symmetry_new, 
            decoder = decoder_with_convs_only,
            encoder_args = encoder_args,
            decoder_args = decoder_args
           )
    
conf.experiment_name = 'Test'
conf.held_out_step = 5

reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

print ae.trainable_parameters()

conv params =  1792
bnorm params =  128
Tensor("Test_1/AvgPool1D/Squeeze:0", shape=(?, 512, 64), dtype=float32) 32768
conv params =  32832
bnorm params =  128
Tensor("Test_1/AvgPool1D_1/Squeeze:0", shape=(?, 128, 64), dtype=float32) 8192
conv params =  65664
bnorm params =  256
Tensor("Test_1/AvgPool1D_2/Squeeze:0", shape=(?, 32, 128), dtype=float32) 4096
conv params =  82048
bnorm params =  256
Tensor("Test_1/AvgPool1D_3/Squeeze:0", shape=(?, 8, 128), dtype=float32) 1024
conv params =  98560
bnorm params =  512
Tensor("Test_1/AvgPool1D_4/Squeeze:0", shape=(?, 4, 256), dtype=float32) 1024
conv params =  196864
bnorm params =  512
Tensor("Test_1/AvgPool1D_5/Squeeze:0", shape=(?, 2, 256), dtype=float32) 512
conv params =  16416
bnorm params =  64
Tensor("Test_1/AvgPool1D_6/Squeeze:0", shape=(?, 1, 32), dtype=float32) 32
Building Decoder
25088
98560
82176
41088
20608
1158
765347


In [29]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
    train_stats = ae.train(train_data, conf, log_file=fout, held_out_data=val_data)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.0751', 'loss=', '0.292327815')
('Epoch:', '0002', 'training time (minutes)=', '0.0673', 'loss=', '0.004921668')
('Epoch:', '0003', 'training time (minutes)=', '0.0655', 'loss=', '0.004149772')
('Epoch:', '0004', 'training time (minutes)=', '0.0623', 'loss=', '0.003802430')
('Epoch:', '0005', 'training time (minutes)=', '0.0619', 'loss=', '0.003610473')
('Held Out Data :', 'forward time (minutes)=', '0.0010', 'loss=', '0.003788688')
('Epoch:', '0006', 'training time (minutes)=', '0.0692', 'loss=', '0.003471919')
('Epoch:', '0007', 'training time (minutes)=', '0.0608', 'loss=', '0.003342077')
('Epoch:', '0008', 'training time (minutes)=', '0.0664', 'loss=', '0.003264538')
('Epoch:', '0009', 'training time (minutes)=', '0.0674', 'loss=', '0.003157743')
('Epoch:', '0010', 'training time (minutes)=', '0.0608', 'loss=', '0.003083504')
('Held Out Data :', 'forward time (minutes)=', '0.0009', 'loss=', '0.003275170')
('Epoch:', '0011', 'training

In [28]:
test_data.point_clouds[0:2].shape
p0 = ae.reconstruct(test_data.point_clouds[0:batch_size])[0]
